## Part I: Data Pre-processing

In [1]:
import pandas as pd

In [2]:
# Download the Google Analogy dataset
!wget http://download.tensorflow.org/data/questions-words.txt

--2025-09-22 02:53:45--  http://download.tensorflow.org/data/questions-words.txt
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.15.251, 172.217.164.27, 172.217.0.91, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.15.251|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘questions-words.txt’

questions-words.txt 100%[===================>] 589.80K  --.-KB/s    in 0.006s  

2025-09-22 02:53:45 (94.9 MB/s) - ‘questions-words.txt’ saved [603955/603955]



In [3]:
# Preprocess the dataset
file_name = "questions-words"
with open(f"{file_name}.txt", "r") as f:
    data = f.read().splitlines()

In [4]:
# check data from the first 10 entries
for entry in data[:10]:
    print(entry)

: capital-common-countries
Athens Greece Baghdad Iraq
Athens Greece Bangkok Thailand
Athens Greece Beijing China
Athens Greece Berlin Germany
Athens Greece Bern Switzerland
Athens Greece Cairo Egypt
Athens Greece Canberra Australia
Athens Greece Hanoi Vietnam
Athens Greece Havana Cuba


In [ ]:
# TODO1: Write your code here for processing data to pd.DataFrame
# Please note that the first five mentions of ": " indicate `semantic`,
# and the remaining nine belong to the `syntatic` category.

In [14]:
# Create the dataframe
df = pd.DataFrame(
    {
        "Question": questions,
        "Category": categories,
        "SubCategory": sub_categories,
    }
)

In [15]:
df.head()

,Question,Category,SubCategory
0,Athens Greece Baghdad Iraq,Semantic,: capital-common-countries
1,Athens Greece Bangkok Thailand,Semantic,: capital-common-countries
2,Athens Greece Beijing China,Semantic,: capital-common-countries
3,Athens Greece Berlin Germany,Semantic,: capital-common-countries
4,Athens Greece Bern Switzerland,Semantic,: capital-common-countries


In [ ]:
df.to_csv(f"{file_name}.csv", index=False)

## Part II: Use pre-trained word embeddings
- After finish Part I, you can run Part II code blocks only.

In [1]:
import pandas as pd
import numpy as np
!pip install gensim
import gensim.downloader
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [3]:
data = pd.read_csv("questions-words.csv")

In [4]:
MODEL_NAME = "glove-wiki-gigaword-100"
# You can try other models.
# https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

# Load the pre-trained model (using GloVe vectors here)
model = gensim.downloader.load(MODEL_NAME)
print("The Gensim model loaded successfully!")

[==================================================] 100.0% 128.1/128.1MB downloaded
The Gensim model loaded successfully!


In [5]:
# Do predictions and preserve the gold answers (word_D)
preds = []
golds = []

for analogy in tqdm(data["Question"]):
      # TODO2: Write your code here to use pre-trained word embeddings for getting predictions of the analogy task.
      # You should also preserve the gold answers during iterations for evaluations later.
      """ Hints
      # Unpack the analogy (e.g., "man", "woman", "king", "queen")
      # Perform vector arithmetic: word_b + word_c - word_a should be close to word_d
      # Source: https://github.com/piskvorky/gensim/blob/develop/gensim/models/keyedvectors.py#L776
      # Mikolov et al., 2013: big - biggest and small - smallest
      # Mikolov et al., 2013: X = vector(”biggest”) − vector(”big”) + vector(”small”).
      """
      # 解析一行題目成四個詞：a b c d
      import re

      s = str(analogy).strip()
      if not s or s.startswith(":"):
          continue

      parts = re.findall(r"[A-Za-z0-9_\-]+", s.lower())

      if len(parts) != 4:
          continue
      a, b, c, d = parts


      # 先把 gold answer 保存起來以便後續評估
      golds.append(d)

      # 用類比向量運算：b + c - a  ≈  d
      # 若有 OOV（不在詞彙表）就回傳 None 以保持 preds/golds 對齊
      try:
          if all(w in model.key_to_index for w in (a, b, c)):
              pred = model.most_similar(positive=[b, c], negative=[a], topn=1)[0][0]
          else:
              pred = None

      except Exception:
          pred = None

      preds.append(pred)

  0%|          | 1/19544 [00:00<45:18,  7.19it/s]

['athens', 'greece', 'baghdad', 'iraq']
['athens', 'greece', 'bangkok', 'thailand']
['athens', 'greece', 'beijing', 'china']
['athens', 'greece', 'berlin', 'germany']


  0%|          | 11/19544 [00:00<09:22, 34.70it/s]

['athens', 'greece', 'bern', 'switzerland']
['athens', 'greece', 'cairo', 'egypt']
['athens', 'greece', 'canberra', 'australia']
['athens', 'greece', 'hanoi', 'vietnam']
['athens', 'greece', 'havana', 'cuba']
['athens', 'greece', 'helsinki', 'finland']
['athens', 'greece', 'islamabad', 'pakistan']
['athens', 'greece', 'kabul', 'afghanistan']
['athens', 'greece', 'london', 'england']
['athens', 'greece', 'madrid', 'spain']


  0%|          | 21/19544 [00:00<07:52, 41.33it/s]

...

['write', 'writes', 'scream', 'screams']
['write', 'writes', 'search', 'searches']
['write', 'writes', 'see', 'sees']
['write', 'writes', 'shuffle', 'shuffles']
['write', 'writes', 'sing', 'sings']
['write', 'writes', 'sit', 'sits']


100%|█████████▉| 19540/19544 [09:55<00:00, 26.07it/s]

['write', 'writes', 'slow', 'slows']
['write', 'writes', 'speak', 'speaks']
['write', 'writes', 'swim', 'swims']
['write', 'writes', 'talk', 'talks']
['write', 'writes', 'think', 'thinks']
['write', 'writes', 'vanish', 'vanishes']


100%|██████████| 19544/19544 [09:55<00:00, 32.82it/s]

['write', 'writes', 'walk', 'walks']
['write', 'writes', 'work', 'works']


In [6]:
# Perform evaluations. You do not need to modify this block!!

def calculate_accuracy(gold: np.ndarray, pred: np.ndarray) -> float:
    return np.mean(gold == pred)

golds_np, preds_np = np.array(golds), np.array(preds)
data = pd.read_csv("questions-words.csv")

# Evaluation: categories
for category in data["Category"].unique():
    mask = data["Category"] == category
    golds_cat, preds_cat = golds_np[mask], preds_np[mask]
    acc_cat = calculate_accuracy(golds_cat, preds_cat)
    print(f"Category: {category}, Accuracy: {acc_cat * 100}%")

# Evaluation: sub-categories
for sub_category in data["SubCategory"].unique():
    mask = data["SubCategory"] == sub_category
    golds_subcat, preds_subcat = golds_np[mask], preds_np[mask]
    acc_subcat = calculate_accuracy(golds_subcat, preds_subcat)
    print(f"Sub-Category{sub_category}, Accuracy: {acc_subcat * 100}%")

Category: Semantic, Accuracy: 65.3399481339497%
Category: Syntactic, Accuracy: 61.255269320843084%
Sub-Category: capital-common-countries, Accuracy: 93.87351778656127%
Sub-Category: capital-world, Accuracy: 88.94783377541998%
Sub-Category: currency, Accuracy: 14.203233256351039%
Sub-Category: city-in-state, Accuracy: 30.806647750304013%
Sub-Category: family, Accuracy: 81.62055335968378%
Sub-Category: gram1-adjective-to-adverb, Accuracy: 24.39516129032258%
Sub-Category: gram2-opposite, Accuracy: 20.073891625615765%
Sub-Category: gram3-comparative, Accuracy: 79.12912912912913%
Sub-Category: gram4-superlative, Accuracy: 54.278074866310156%
Sub-Category: gram5-present-participle, Accuracy: 69.50757575757575%
Sub-Category: gram6-nationality-adjective, Accuracy: 87.86741713570981%
Sub-Category: gram7-past-tense, Accuracy: 55.44871794871795%
Sub-Category: gram8-plural, Accuracy: 71.996996996997%
Sub-Category: gram9-plural-verbs, Accuracy: 58.39080459770115%


In [12]:
# Collect words from Google Analogy dataset
SUB_CATEGORY = ": family"

# TODO3: Plot t-SNE for the words in the SUB_CATEGORY `: family`

# 取出 : family 子分類的四詞題目
import re, numpy as np
from sklearn.manifold import TSNE

def collect_pairs(dataframe, subcat):
    """Return list of [a,b,c,d] pairs under the given sub-category."""
    pairs = []

    # Case A: a raw Mikolov-like column named "Question" (lines with ":" headers)
    if "Question" in dataframe.columns:
        lines = list(map(str, dataframe["Question"]))
        collect = False
        for s in lines:
            s = s.strip()
            if not s:
                continue
            if s.startswith(":"):
                collect = (s.lower() == subcat.lower())
                continue
            if collect:
                toks = re.findall(r"[A-Za-z0-9_\-]+", s.lower())
                if len(toks) == 4:
                    pairs.append(toks)

    # Case B: already split into columns a,b,c,d (optionally with SubCategory)
    if not pairs and {"a","b","c","d"}.issubset(set(dataframe.columns)):
        df = dataframe
        if "SubCategory" in df.columns:
            df = df[df["SubCategory"].astype(str).str.strip().str.lower()
                          .eq(subcat.lower())]
        for _, r in df.iterrows():
            a, b, c, d = map(lambda x: str(x).lower(), (r["a"], r["b"], r["c"], r["d"]))
            pairs.append([a, b, c, d])

    return pairs

# 1) 取出該子分類的四詞題目
pairs = collect_pairs(data, SubCategory)

# 2) 蒐集會用到、且存在於詞表的詞
words = sorted({w for a,b,c,d in pairs for w in (a,b,c,d) if w in model.key_to_index})

# 安全防呆：若抓不到任何詞，直接說明原因
if not words:
    raise ValueError(f"No words collected for {SubCategory}. "
                     "Check your DataFrame schema (raw 'Question' lines vs. columns a,b,c,d) "
                     "and make sure tokens exist in the model vocabulary.")

# 3) 準備向量（注意：必須是 NumPy 陣列，而非 list）
X = np.vstack([model[w] for w in words]).astype(np.float32)

# 4) t-SNE（perplexity 必須 < 樣本數）
n = len(words)
perp = min(30, max(2, n - 1))
tsne = TSNE(n_components=2, init="random", learning_rate="auto",
            perplexity=perp, random_state=0)
Y = tsne.fit_transform(X)  # shape: (n, 2)

# 5) 畫圖（簡潔款）
plt.figure(figsize=(8, 6))
plt.scatter(Y[:, 0], Y[:, 1], s=20)
for (x, y), w in zip(Y, words):
    plt.annotate(w, (x, y), xytext=(2, 2), textcoords="offset points", fontsize=9)

plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
# （你原本的 plt.title / plt.show / plt.savefig 會接著執行）

plt.title("Word Relationships from Google Analogy Task")
plt.show()
plt.savefig("word_relationships.png", bbox_inches="tight")

NameError: name 'SubCategory' is not defined

### Part III: Train your own word embeddings

### Get the latest English Wikipedia articles and do sampling.
- Usually, we start from Wikipedia dump (https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2). However, the downloading step will take very long. Also, the cleaning step for the Wikipedia corpus ([`gensim.corpora.wikicorpus.WikiCorpus`](https://radimrehurek.com/gensim/corpora/wikicorpus.html#gensim.corpora.wikicorpus.WikiCorpus)) will take much time. Therefore, we provide cleaned files for you.

In [8]:
# Download the split Wikipedia files
# Each file contain 562365 lines (articles).
!gdown --id 1jiu9E1NalT2Y8EIuWNa1xf2Tw1f1XuGd -O wiki_texts_part_0.txt.gz
!gdown --id 1ABblLRd9HXdXvaNv8H9fFq984bhnowoG -O wiki_texts_part_1.txt.gz
!gdown --id 1z2VFNhpPvCejTP5zyejzKj5YjI_Bn42M -O wiki_texts_part_2.txt.gz
!gdown --id 1VKjded9BxADRhIoCzXy_W8uzVOTWIf0g -O wiki_texts_part_3.txt.gz
!gdown --id 16mBeG26m9LzHXdPe8UrijUIc6sHxhknz -O wiki_texts_part_4.txt.gz

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1jiu9E1NalT2Y8EIuWNa1xf2Tw1f1XuGd

but Gdown can't. Please check connections and permissions.
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warn

In [ ]:
# Download the split Wikipedia files
# Each file contain 562365 lines (articles), except the last file.
!gdown --id 17JFvxOH-kc-VmvGkhG7p3iSZSpsWdgJI -O wiki_texts_part_5.txt.gz
!gdown --id 19IvB2vOJRGlrYulnTXlZECR8zT5v550P -O wiki_texts_part_6.txt.gz
!gdown --id 1sjwO8A2SDOKruv6-8NEq7pEIuQ50ygVV -O wiki_texts_part_7.txt.gz
!gdown --id 1s7xKWJmyk98Jbq6Fi1scrHy7fr_ellUX -O wiki_texts_part_8.txt.gz
!gdown --id 17eQXcrvY1cfpKelLbP2BhQKrljnFNykr -O wiki_texts_part_9.txt.gz
!gdown --id 1J5TAN6bNBiSgTIYiPwzmABvGhAF58h62 -O wiki_texts_part_10.txt.gz

In [ ]:
# Extract the downloaded wiki_texts_parts files.
!gunzip -k wiki_texts_part_*.gz

In [ ]:
# Combine the extracted wiki_texts_parts files.
!cat wiki_texts_part_*.txt > wiki_texts_combined.txt

In [ ]:
# Check the first ten lines of the combined file
!head -n 10 wiki_texts_combined.txt

Please note that we used the default parameters of [`gensim.corpora.wikicorpus.WikiCorpus`](https://radimrehurek.com/gensim/corpora/wikicorpus.html#gensim.corpora.wikicorpus.WikiCorpus) for cleaning the Wiki raw file. Thus, words with one character were discarded.

In [ ]:
# Now you need to do sampling because the corpus is too big.
# You can further perform analysis with a greater sampling ratio.

import random

wiki_txt_path = "wiki_texts_combined.txt"
# wiki_texts_combined.txt is a text file separated by linebreaks (\n).
# Each row in wiki_texts_combined.txt indicates a Wikipedia article.

with open(wiki_txt_path, "r", encoding="utf-8") as f:
    with open(output_path, "w", encoding="utf-8") as output_file:
    # TODO4: Sample `20%` Wikipedia articles
    # Write your code here

In [ ]:
# TODO5: Train your own word embeddings with the sampled articles
# https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec
# Hint: You should perform some pre-processing before training.

In [ ]:
data = pd.read_csv("questions-words.csv")

In [ ]:
# Do predictions and preserve the gold answers (word_D)
preds = []
golds = []

for analogy in tqdm(data["Question"]):
      # TODO6: Write your code here to use your trained word embeddings for getting predictions of the analogy task.
      # You should also preserve the gold answers during iterations for evaluations later.
      """ Hints
      # Unpack the analogy (e.g., "man", "woman", "king", "queen")
      # Perform vector arithmetic: word_b + word_c - word_a should be close to word_d
      # Source: https://github.com/piskvorky/gensim/blob/develop/gensim/models/keyedvectors.py#L776
      # Mikolov et al., 2013: big - biggest and small - smallest
      # Mikolov et al., 2013: X = vector(”biggest”) − vector(”big”) + vector(”small”).
      """

In [ ]:
# Collect words from Google Analogy dataset
SUB_CATEGORY = ": family"

# TODO7: Plot t-SNE for the words in the SUB_CATEGORY `: family`


plt.title("Word Relationships from Google Analogy Task")
plt.show()
plt.savefig("word_relationships.png", bbox_inches="tight")